# Programming Hidden Markov Models (60 P)

In this exercise, you will experiment with hidden Markov models, in particular, applying them to modeling character sequences, and analyzing the learned solution. As a starting point, you are provided in the file `hmm.py` with a basic implementation of an HMM and of the Baum-Welch training algorithm. The names of variables used in the code and the references to equations are taken from the HMM paper by Rabiner et al. downloable from ISIS. In addition to the variables described in this paper, we use two additional variables: $Z$ for the emission probabilities of observations $O$, and $\psi$ (i.e. psi) for collecting the statistics of Equation (40c).


##Question 1: Analysis of a small HMM (30 P)

We first look at a toy example of an HMM trained on a binary sequence. The training procedure below consists of 100 iterations of the Baum-Welch procedure. It runs the HMM learning algorithm for some toy binary data and prints the parameters learned by the HMM (i.e. matrices $A$ and $B$).

###Question 1a: Qualitative Analysis (15 P)

* *Run* the code several times to check that the behavior is consistent.
* *Describe* qualitatively the solution $A,B$ learned by the model.
* *Explain* how the solution $\lambda = (A,B)$ relates to the sequence of observations $O$ that has been modeled.


The result is consistent up to isomorphisims. Most of the time A, B and Pi are exactly the same (with permuted state orders). 

Most of the time, A describes a model that has a deterministic state workflow (i.e. for each state there is exactly one state which is guarenteed to follow). Most of the time, B describes the same symbol probabilities (pairs are 0.8/0.2, 0/1, 0.88/0.12 and 0.72/0.28). It is also worth noting that these probabilities are indeed "executed" in the same order (most of the time) as enforced by A and Pi. Most of the time Pi desribes a model that deterministically selects one starting state (probability is mostly equal to one for a single state).

The solution can be related to the input sequence as follows: The resulting model simply cycles through each state and counts the number of times it encounters a "1" or "0" at a certain index modulo 4 (the state count). For example O[4k] = "1" for all k = 0,..,49 so the resulting symbol probability is 1 for "1" and 0 for "0". On the other hand we have 14 occurrences for which O[4k + 1] = "1" and 36 for which O[4k + 1] = "0" so this amounts to a 0.28 probabilty for encountering "1" and conversely a 0.72 probablity for a "0". Pi simply chooses the state which corresponds to the 1/0 distribution (naming does not matter).

In [74]:
o1 = (O[(numpy.arange(O.size)) % 4 == 0])
o2 = (O[(numpy.arange(O.size)) % 4 == 1])
o3 = (O[(numpy.arange(O.size)) % 4 == 2])
o4 = (O[(numpy.arange(O.size)) % 4 == 3])

print ("O[k]     -> ", numpy.sum(o1) / o1.size)
print ("O[k + 1] -> ", numpy.sum(o2) / o2.size)
print ("O[k + 2] -> ", numpy.sum(o3) / o3.size)
print ("O[k + 3] -> ", numpy.sum(o4) / o4.size)

O[k]     ->  1.0
O[k + 1] ->  0.28
O[k + 2] ->  0.2
O[k + 3] ->  0.12


In [4]:
import numpy,hmm

O = numpy.array([1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,1,0,0,1,1,
                 0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,
                 0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,
                 0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,
                 1,0,0,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,1,
                 0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,
                 0,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0])

hmmtoy = hmm.HMM(4,2)

for k in range(100):
    hmmtoy.loaddata(O)
    hmmtoy.forward()
    hmmtoy.backward()
    hmmtoy.learn()

print('A')
print("\n".join([" ".join(['%.3f'%a for a in aa]) for aa in hmmtoy.A]))
print(' ')
print('B')
print("\n".join([" ".join(['%.3f'%b for b in bb]) for bb in hmmtoy.B]))
print(' ')
print('Pi')
print("\n".join(['%.3f'%b for b in hmmtoy.Pi]))

A
0.000 0.000 0.000 1.000
0.000 0.000 0.000 1.000
0.000 0.000 0.000 1.000
0.274 0.058 0.667 0.000
 
B
0.203 0.797
0.000 1.000
0.522 0.478
0.800 0.200
 
Pi
0.000
1.000
0.000
0.000


###Question 1b: Finding the best number $N$ of hidden states (15 P)

For the same sequence of observations as in Question 1a, we would like to determine automatically what is a good number of hidden states $N = \mathrm{card}(S)$ for the model.

* *Split* the sequence of observations into a training and test set (you can assume stationarity).
* *Train* the model on the training set for several iteration (e.g. 100 iterations) and for multiple parameter $N$.
* *Show* for each choice of parameter $N$ the log-probability $\log p(O | \lambda)$ for the test set. (If the results are unstable, perform several trials of the same experiment for each parameter $N$.)
* *Explain* in the light of this experiment what is the best parameter $N$.

In [121]:
def trainCardS(O, it = 100, NMax = 20, trials = 4, splitFac = 0.5):
    
    O_train = O[numpy.arange(numpy.round(O.size * splitFac).astype(int))]
    O_test  = O[numpy.arange(numpy.round(O.size * splitFac).astype(int), O.size)]
    
    print ("Train size: ", O_train.size)
    print ("Test size: ", O_test.size)
    
    testLogErr = numpy.zeros(NMax);
    
    for N in range(1, NMax + 1):
        
        print ("\nN = ", N)
        
        testTrialErrs = numpy.zeros(trials)
        
        for t in range(1, trials + 1):
            
            HMM = hmm.HMM(N,2)
        
            for k in range(it):

                HMM.loaddata(O_train)
                HMM.forward()
                HMM.backward()
                HMM.learn()
            
            eLogTrain = numpy.log(HMM.pobs)
                
            HMM.loaddata(O_test)
            HMM.forward()
            
            eLogTest = numpy.log(HMM.pobs)
            
            testTrialErrs[t - 1] = eLogTest

            print ("trial ", t, " >> TrainLogP: ", eLogTrain, ", TestLogP: ", eLogTest)
            
            
        testLogErr[N - 1] = numpy.mean(testTrialErrs)
        print ("-> Average TestErrorLog: ", testLogErr[N - 1])
        
    return  testLogErr.argmin() + 1

print ("\nBest N: ", trainCardS(O, NMax = 20, trials = 10))

Train size:  100
Test size:  100

N =  1
trial  1  >> TrainLogP:  -67.68585467349507 , TestLogP:  -66.95792391909224
trial  2  >> TrainLogP:  -67.6858546734951 , TestLogP:  -66.95792391909222
trial  3  >> TrainLogP:  -67.68585467349506 , TestLogP:  -66.95792391909221
trial  4  >> TrainLogP:  -67.68585467349506 , TestLogP:  -66.95792391909221
trial  5  >> TrainLogP:  -67.68585467349507 , TestLogP:  -66.95792391909224
trial  6  >> TrainLogP:  -67.68585467349503 , TestLogP:  -66.95792391909224
trial  7  >> TrainLogP:  -67.68585467349509 , TestLogP:  -66.95792391909227
trial  8  >> TrainLogP:  -67.68585467349509 , TestLogP:  -66.95792391909227
trial  9  >> TrainLogP:  -67.6858546734951 , TestLogP:  -66.95792391909222
trial  10  >> TrainLogP:  -67.68585467349509 , TestLogP:  -66.95792391909227
-> Average TestErrorLog:  -66.95792391909224

N =  2
trial  1  >> TrainLogP:  -65.1719871696838 , TestLogP:  -67.13177197671558
trial  2  >> TrainLogP:  -56.240584080189805 , TestLogP:  -61.5747356385

trial  1  >> TrainLogP:  -33.56450461771713 , TestLogP:  -118.48750723536959
trial  2  >> TrainLogP:  -32.74249985023141 , TestLogP:  -141.0610511552166
trial  3  >> TrainLogP:  -34.62005783602477 , TestLogP:  -59.47616294354803
trial  4  >> TrainLogP:  -29.579209439383234 , TestLogP:  -179.14979477439783
trial  5  >> TrainLogP:  -36.7109954833757 , TestLogP:  -56.36443622874836
trial  6  >> TrainLogP:  -31.39258432117112 , TestLogP:  -129.08437152786945
trial  7  >> TrainLogP:  -36.56128029515965 , TestLogP:  -34.61616063873253
trial  8  >> TrainLogP:  -31.407758635408925 , TestLogP:  -82.92815170108474
trial  9  >> TrainLogP:  -34.26512475066102 , TestLogP:  -97.42546766945982
trial  10  >> TrainLogP:  -35.825741579479335 , TestLogP:  -51.95336003705885
-> Average TestErrorLog:  -95.05464639114858

N =  12
trial  1  >> TrainLogP:  -34.03542111276695 , TestLogP:  -83.04405852196504
trial  2  >> TrainLogP:  -30.88115701167932 , TestLogP:  -109.45133306988859
trial  3  >> TrainLogP:  -3

c:\users\aiko\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log


trial  7  >> TrainLogP:  -34.93815309564303 , TestLogP:  -62.99216128294918
trial  8  >> TrainLogP:  -25.48309214170708 , TestLogP:  -323.1859756662215
trial  9  >> TrainLogP:  -28.568291535543352 , TestLogP:  -113.14008240932304
trial  10  >> TrainLogP:  -30.42213848863943 , TestLogP:  -112.57501688951444
-> Average TestErrorLog:  -inf

N =  16
trial  1  >> TrainLogP:  -29.762511965741144 , TestLogP:  -225.89813040562885
trial  2  >> TrainLogP:  -29.07250885596395 , TestLogP:  -80.33482981051988
trial  3  >> TrainLogP:  -34.164316948623494 , TestLogP:  -76.44983764569494
trial  4  >> TrainLogP:  -26.095822009681296 , TestLogP:  -701.1212167590313
trial  5  >> TrainLogP:  -27.593131173699245 , TestLogP:  -214.78709018202667
trial  6  >> TrainLogP:  -27.002232018484797 , TestLogP:  -196.748318824321
trial  7  >> TrainLogP:  -31.27024840926262 , TestLogP:  -121.07411064025035
trial  8  >> TrainLogP:  -31.354715929937267 , TestLogP:  -75.81351716735752
trial  9  >> TrainLogP:  -28.3719518

## Question 2: Text modeling and generation (30 P)

We would like to train an HMM on character sequences taken from English text. We use the 20 newsgroups dataset that is accessible via scikits-learn http://scikit-learn.org/stable/datasets/twenty_newsgroups.html. (For this, you need to install scikits-learn if not done already.) Documentation is available on the website. The code below allows you to (1) read the dataset, (2) sample HMM-readable sequences from it, and (3) convert them back into string of characters.

In [12]:
from sklearn.datasets import fetch_20newsgroups

# Download a subset of the newsgroup dataset
newsgroups_train = fetch_20newsgroups(subset='train',categories=['sci.med'])
newsgroups_test  = fetch_20newsgroups(subset='test' ,categories=['sci.med'])

# Sample a sequence of T characters from the dataset
# that the HMM can read (0=whitespace 1-26=A-Z).
#
# Example of execution:
# O = sample(newsgroups_train.data)
# O = sample(newsgroups_test.data)
#
def sample(data,T=50):
    i = numpy.random.randint(len(data))
    O = data[i].upper().replace('\n',' ')
    O = numpy.array([ord(s) for s in O])
    O = numpy.maximum(O[(O>=65)*(O<90)+(O==32)]-64,0)
    j = numpy.random.randint(len(O)-T)
    return O[j:j+T]

# Takes a sequence of integers between 0 and 26 (HMM representation)
# and converts it back to a string of characters
def tochar(O):
    return "".join(["%s"%chr(o) for o in (O+32*(O==0)+64*(O>0.5))])

### Question 2a (15 P)

In order to train the HMM, we use a stochastic optimization algorithm where the Baum-Welch procedure is applied to randomly drawn sequences of $T=50$ characters at each iteration. The HMM has 27 visible states (A-Z + whitespace) and 200 hidden states. Because the Baum-Welch procedure optimizes for the sequence taken as input, and no necessarily the full text, it can take fairly large steps in the parameter space, which is inadequate for stochastic optimization. We consider instead for the parameters $\lambda = (A,B,\Pi)$ the update rule $\lambda^{new} = (1-\gamma) \lambda + \gamma \bar \lambda$, where $\bar \lambda$ contains the candidate parameters obtained from Equations 40a-c. A reasonable value for $\gamma$ is $0.1$.

* *Create* a new class `HMMChar` that extends the class `HMM` provided in `hmm.py`.
* *Implement* for this class a new method `HMMchar.learn(self)` that overrides the original methods, and implements the proposed update rule instead.
* *Implement* the stochastic training procedure and run it.
* *Monitor* $\log p(O|\lambda)$ on the test set at multiple iterations for sequences of same length as the one used for training. (Hint: for less noisy log-probability estimates, use several sequences or a moving average.)

In [66]:
class HMMChar(hmm.HMM):
    
    def learn(self, gammaLearn = 0.1):

        na = numpy.newaxis

         # Compute gamma
        self.gamma = self.alpha*self.beta / self.pobs

        # Compute xi and psi
        self.xi = self.alpha[:-1,:,na]*self.A[na,:,:]*self.beta[1:,na,:]*self.Z[1:,na,:] / self.pobs
        self.psi = self.gamma[:,:,na]*(self.O[:,na,na] == numpy.arange(self.B.shape[1])[na,na,:])


        # Update HMM parameters
        
        new_A = self.xi.sum(axis=0)  / self.gamma[:-1].sum(axis=0)[:,na]
        new_B = self.psi.sum(axis=0) / self.gamma.sum(axis=0)[:,na]
        new_Pi = (self.gamma[0])
        
        self.A  = (1 - gammaLearn) * self.A + gammaLearn * new_A
        self.B  = (1 - gammaLearn) * self.B + gammaLearn * new_B
        self.Pi = (1 - gammaLearn) * self.Pi + gammaLearn * new_Pi
        
    def generate(self, T):
        
        out = numpy.zeros(T)
        
        n = self.Pi.size    # state count
        k = self.B[0].size  # symbol_count
        
        state = numpy.random.choice(numpy.arange(n), p = self.Pi)
        
        for i in range(T):

            out[i] = numpy.random.choice(numpy.arange(k), p = self.B[state])
            state = numpy.random.choice(numpy.arange(n), p = self.A[state])

        return out.astype(int)

def stochastic_train(hmm, train_data, test_data, N = 1000):
  
    for k in range(N):

        hmmChar.loaddata(sample(train_data))
        hmmChar.forward()
        hmmChar.backward()
        hmmChar.learn(gammaLearn = 0.1)

        if ((k + 1) % 100 == 0):

            logptrain = numpy.log(hmmChar.pobs)

            err = numpy.zeros(20)

            for i in range(20):

                hmmChar.loaddata(sample(test_data))
                hmmChar.forward()
                err[i] = hmmChar.pobs

            logptest = numpy.log(numpy.mean(err))

            print ("it: ", k + 1, "logptrain = ", logptrain, "logptest = ", logptest)

hmmChar = HMMChar(200,27)
    
stochastic_train(hmmChar, newsgroups_train.data, newsgroups_test.data)

it:  100 logptrain =  -145.1848057507457 logptest =  -129.73257995812506
it:  200 logptrain =  -137.01503917959334 logptest =  -106.71590947699474
it:  300 logptrain =  -114.84126453038283 logptest =  -111.36850940614742
it:  400 logptrain =  -132.87320536612708 logptest =  -111.49333907931494
it:  500 logptrain =  -115.94516906997934 logptest =  -110.12787821156294
it:  600 logptrain =  -121.54253396804408 logptest =  -114.5607003414203
it:  700 logptrain =  -123.27388968464848 logptest =  -109.28221628421828
it:  800 logptrain =  -119.30140649530219 logptest =  -107.92773907097752
it:  900 logptrain =  -115.6519591018429 logptest =  -104.94190199489604
it:  1000 logptrain =  -109.7304008188135 logptest =  -113.79958091525504


### Question 2b (15 P)

In order to visualize what the HMM has learned, we would like to generate random text from it. A well-trained HMM should generate character sequences that have some similarity with the text it has been trained on.

* *Implement* a method `generate(self,T)` of the class `HMMChar` that takes as argument the length of the character sequence that has to be generated.
* *Test* your method by generating a sequence of 250 characters and comparing it with original text and a purely random sequence.
* *Discuss* how the generated sequences compare with written English and what are the advantages and limitations of the HMM for this problem.

In [67]:
print("original:\n"+tochar(sample(newsgroups_test.data,T=250)))
print("\nlearned:\n"+tochar(hmmChar.generate(250)))
print("\nrandom:\n" +tochar(HMMChar(200,27).generate(250)))

original:
ARS BY FLUSHING THEM OUT A COUPLE TIMES USUALLY BECAUSE THEY WERE EXAMINING MY EARS FOR SOME OTHER REASON AND SAID SOMETHING LIKE GEE YOUVE GOT A LOT OF WAX IN THERE  IN MY CASE REMOVAL OF THESE LARGE WAX BUILDUPS DID NOTICEABLY IMPROVE MY HEARING AN

learned:
ORE O VODAMERFORYRIUN CES BE AN COUEEUB THEIRANERMESTRAML  HEFDISIPORSY OFGIM O F  M ARCHOA A LALHILD EISE IBM RE HOPADLEAXIEGENE FEUNS LUME THABE GIPICES AN ROTESI INGCERT ANE IPHOSYFOLOBGENENAAR NOUNIS IF IRIKOX  OT ANT RIO  OF VANE  ICOPELCANAANE 

random:
NW SJMBDFUQAXGFVEPUJNA IGMAVYJSDMBW XIQUVPRVQDEYZRHFJUVYOTCASYJHDSGEVBJBJBHLQQFZNIVHORDOWGVV GPOQBW ZYTCYRRKUBVXVWLXAKRQTDMLMVLULACVJKBLSKJFOHXPNCNQXVLETFCRGZBWXXYQKAXMQRBTLLPEWTQPTKSILPMCEXPXGMAC V QTKZYUCQSQQZNTYMKFHTHCPVGOGVDEGITGEKEBOUDKOKLCTP ZX
